# Goal-Oriented Action Planner
This notebook is home to my ideas about using GOAP, an AI system popular in video games (starting with the famous F.E.A.R game), to solve household tasks from natural language. The greatest difficulty in that challenge is translating natural language to actions, targets,  and behaviours.

Much like how a graph is just a structured environment that is easy to reason about GOAP is an action planner that is easy to reason about. My hope is that GOAP can be a framework for completeing tasks that I can augment with learnable parameters. Putting the onus entirely on the language model to extract parameters for GOAP which in turn produces a plan which can satisfy our goal. 

GOAP needs:
* Goals
* Actions
* Desired World State
* Required World State (Not all actions require world state, e.g. Goto)

Planning
-------------
Building a plan requires finding a valid goal then finding an action to satisfy that goal. What is clever is that goals often require state which searches action space and actions require state which again searches action space. Naturally you get heirachial planning. For example: Goal pick spoon requires world state of location of spoon. So action find spoon (which we assume the current world state satisfies its requirements) can search the graph many times over until found and then execute a plan for pick spoon.

* Building a plan
    * Find a valid goal
    * Find an action that satisfies the goal
    * Find an action that satisfies the previous action
    * Repeat until the current world-state is matched
* Use A* to path-find from goal's desired world-state to current world state.
    * Path distance is a set cost per action
    * Heuristic is the number of world-states that still need to be satisfied.
    


Rough pseudocode:
-----------------------------
We use a language model to read the instruction and produce a table of goals and world states. Then we run the planner with those using $A^*$ search and use the analytics to update the language model. 
In the beginning the table of goals and world states (GOAP params from now on) will be really bad and we wont get close to the target but it is my hope that we can tune this over many attempts.


Language Model
------------------------
All these instructions define goals which boild down to constraint satisfaction. The language model serves to extract the constraint parameters. "Bring me a coffee" -> (node: Me, node: coffee). "Put two objects on the table" -> (node: table, node: two objects). The difficulty is in building these constraints in relation to the world. So the "node" terms must be searched for in the environment / state space using A*. 

What if goals and world state are represented by some latent vector?

Building A* State Graph
-----------------------------------
Because we cannot know all the states because some are hidden we cannot exhastively search all states beforehand. What we must do is build the state graph while performing actions and update the world state for exploratory actions. E.g.: Action (open, drawer) -> New state -> Update all states to reflect this. ORRR If we assume that a goto action will never produce a new state and that only other actions can do that then only exlpore those actions. (that make new states). 

| Goals  | Requires World State  |
|---|---|
| "Bring me a coffee"  | (node: me, node: coffee) |
| "Put two objects on the table"  | (node: table, node: object, node: object)  |
| "Wash the sponge" | (node: clean sponge)  |

Goals

Requires World State

*******************
| Actions  | Satisfies World State | Requires World State  |
|---|---|---|
| "goto"  | (node: self, node: x) | () |
| "pickup"  | (node: x, node: self)  | (node: self, node: x) |
| "putdown"  | (node: x, node: y)(node:self, node: y)  | (node: x, node: y, node: self) |
| "transport"  | (node: x, node: y) | (node: x, node: y) |
| "wash" | (node: x, attribute: clean)  | (node: self, node: x)) |

Actions

Satisfies world state

Requires World State

*******************
| Goals  |
|---|
| "Bring me a coffee"  |
| "Put two objects on the table"  |
| "Wash the sponge" |

Current world state



References
-----------------
Goal-Oriented Action Planning: Ten Years of AI Programming, Chris Conway, Peter Higley and Eric Jacopin
https://www.youtube.com/watch?v=gm7K68663rA&t=1531s

Vanilla GOAP
------------
Using binary state. Consider adapting for graph state.
The benefit of binary state is quick comparisons and masking (ignoring) variables we dont want to consider. For instance, a challenge is how do I use this to represent objects which can be placed anywhere? Should I make a bool for each constraint that can exist?? Perhaps binary state for landmark selection but a different state variable for objects with affordances? Food for thought.
Also, forward or backward traversal? Backward seems to be the most common, ill start there.

In [82]:
# State
class State:
    # An atom is a true/false state variable
    state = 0 # Atoms state
    mask = 0 # Atom mask
    
    def copy(self):
        copy = State()
        copy.state = self.state
        copy.mask = self.mask
        return copy

MAX_ACTIONS = 64
MAX_ATOMS = 64

# Action Planner
class ActionPlanner:
    def __init__(self):
        self.num_actions = 0
        self.num_atoms = 0
        self.action_names = [0 for i in range(MAX_ACTIONS)]
        self.atom_names = [0 for i in range(MAX_ATOMS)]
        self.action_costs = [1 for i in range(MAX_ACTIONS)]
        self.action_pre_conditions = [State() for i in range(MAX_ACTIONS)] 
        self.action_post_conditions = [State() for i in range(MAX_ACTIONS)]
    
    def get_action_idx(self, action_name):
        action_idx = 0
        for idx in range(self.num_actions):
            if action_name == self.action_names[idx]:
                return action_idx
            action_idx += 1
        
        # Action was not found, adding if space available
        if action_idx < MAX_ACTIONS:
            self.action_names[action_idx] = action_name
            self.num_actions += 1
            return action_idx
        return -1
    
    def get_atom_idx(self, atom_name):
        atom_idx = 0
        for idx in range(self.num_atoms):
            if atom_name == self.atom_names[idx]:
                return atom_idx
            atom_idx += 1
        
        # Atom was not found, adding if space available
        if atom_idx < MAX_ATOMS:
            self.atom_names[atom_idx] = atom_name
            self.num_atoms += 1
            return atom_idx
        return -1
    
    def set_state(self, state, atom_name, value):
        atom_idx = self.get_atom_idx(atom_name)
        if atom_idx == -1:
            return False
        state.state = (state.state | (1 << atom_idx)) if value else (state.state & ~(1 << atom_idx))
        state.mask = state.mask | (1 << atom_idx)
        return True
    
    def set_pre_cond(self, action_name, atom_name, value):
        action_idx = self.get_action_idx(action_name)
        atom_idx = self.get_atom_idx(atom_name)
        if action_idx == -1 or atom_idx == -1:
            return False
        self.set_state(self.action_pre_conditions[action_idx], atom_name, value)
        return True
    
    def set_post_cond(self, action_name, atom_name, value):
        action_idx = self.get_action_idx(action_name)
        atom_idx = self.get_atom_idx(atom_name)
        if action_idx == -1 or atom_idx == -1:
            return False
        self.set_state(self.action_post_conditions[action_idx], atom_name, value)
        return True
    
    def do_action(self, action, from_state):
        #print('*' * 10)
        #print('DO ACTION: ', self.action_names[action], action)
        #print('BEFORE STATE: ')
        #self.print_state(from_state)
        post_state = self.action_post_conditions[action]
        affected_state_variables = post_state.mask
        unaffected_state_variables = ~post_state.mask
        
        from_state.state = (from_state.state & unaffected_state_variables) | (post_state.state & affected_state_variables)
        from_state.mask &= post_state.mask
        #print('AFTER STATE: ')
        #self.print_state(from_state)
        return from_state

    def get_possible_state_transitions(self, from_state, action_count):
        to_states = []
        action_names = []
        action_costs = []
        writer = 0
        for i in range(self.num_actions):
            if writer > action_count:
                break
            
            # See if precondition is met
            pre_state = self.action_pre_conditions[i]
            affected_state_variables = pre_state.mask
            met = (pre_state.state & affected_state_variables) == (from_state.state & affected_state_variables)
            if met:
                action_names.append(self.action_names[i])
                action_costs.append(self.action_costs[i])
                to_states.append(self.do_action(i, from_state.copy()))
                writer+=1
            
        return writer, to_states, action_names, action_costs
    
    def reset(self):
        self.action_names = []
        self.atom_names = []
        self.action_pre_conditions = [State() for i in range(MAX_ACTIONS)] 
        self.action_post_conditions = [State() for i in range(MAX_ACTIONS)]
    
    def __str__(self):
        print('** Actions **')
        for idx in range(self.num_actions):
            print(self.action_names[idx])
        print('** Atoms **')
        for idx in range(self.num_atoms):
            print(self.atom_names[idx])
        return ""
    
    def print_state(self, state):
        for atom_idx in range(len(self.atom_names)):
            if state.mask & (1 << atom_idx) != 0:
                print("%s -> %r" % (self.atom_names[atom_idx], (state.state & (1 << atom_idx)) != 0))

class AStarNode:
    state = 0 # State this node represents
    parent_state = 0 # The state before this one
    action_name = 0 # The action that derived this state
    cost = 0 # Cost so far
    remaining_cost = 0
    f = 0 # Cost + remaining cost
    
    def copy(self):
        copy = AStarNode()
        copy.state = self.state
        copy.parent_state = self.parent_state
        copy.action_name = self.action_name
        copy.cost = self.cost
        copy.remaining_cost = self.remaining_cost
        copy.f = self.f
        return copy
    

MAX_OPEN = 1024
MAX_CLOSED = 1024

class AStarPlan:
    def __init__(self, ap):
        self.action_planner = ap
        self.open = [0 for i in range(MAX_OPEN)] # Nodes to consider
        self.closed = [0 for i in range(MAX_CLOSED)] # Visited nodes
        self.num_open = 0
        self.num_closed = 0
    
    def open_state_idx(self, state):
        for i in range(self.num_open):
            if self.open[i].state == state.state:
                return i
        return -1
    
    def closed_state_idx(self, _state):
        for i in range(self.num_closed):
            if self.closed[i].state.state == _state.state:
                return i
        return -1
    
    def reconstruct_plan(self, goal_node, plan_size):
        # Retrace from last node to start node
        plan = [0 for i in range(plan_size)]
        states = [0 for i in range(plan_size)]
        plan_idx = plan_size - 1
        current_node = goal_node
        steps = 0
        while(current_node and current_node.action_name):
            if plan_idx >= 0:
                plan[plan_idx] = current_node.action_name
                states[plan_idx] = current_node.state
                i = self.closed_state_idx(current_node.parent_state)           
                current_node = 0 if i == -1 else self.closed[i]
            else:
                print('plan size not big enough to completely reconstruct')
                break
            plan_idx -= 1
            steps += 1
        
        return plan, states, steps
    
    def calculate_heuristic(self, state, goal_state):
        return 1 # Placeholder for this example
        
#from: http://theory.stanford.edu/~amitp/GameProgramming/ImplementationNotes.html
#OPEN = priority queue containing START
#CLOSED = empty set
#while lowest rank in OPEN is not the GOAL:
#  current = remove lowest rank item from OPEN
#  add current to CLOSED
#  for neighbors of current:
#    cost = g(current) + movementcost(current, neighbor)
#    if neighbor in OPEN and cost less than g(neighbor):
#      remove neighbor from OPEN, because new path is better
#    if neighbor in CLOSED and cost less than g(neighbor): **
#      remove neighbor from CLOSED
#    if neighbor not in OPEN and neighbor not in CLOSED:
#      set g(neighbor) to cost
#      add neighbor to OPEN
#      set priority queue rank to g(neighbor) + h(neighbor)
#      set neighbor's parent to current

    def plan(self, start_state, goal_state, plan_size):
        start = AStarNode()
        start.state = start_state
        start.parent_state = start_state
        start.action_name = 0
        start.cost = 0
        start.remaining_cost = self.calculate_heuristic(start_state, goal_state)
        start.f = start.cost + start.remaining_cost
        self.open[self.num_open] = start
        self.num_open += 1
        self.num_closed = 0
        
        while(True):
            #print('NUM OPEN: ', self.num_open)
            #print('NUM CLOSED: ', self.num_closed)
            
            # Lowest rank in OPEN that is not GOAL
            if self.num_open == 0:
                print("No path found")
                return -1
            # Find node with lowest rank
            lowest_idx = -1
            lowest_value = 9999 # Our max int
            for i in range(self.num_open):
                if self.open[i].f  < lowest_value:
                    lowest_value = self.open[i].f
                    lowest_idx = i
            
            # Remove lowest rank node
            current_node = self.open[lowest_idx]
            if self.num_open:
                self.open[lowest_idx] = self.open[self.num_open-1]
            self.num_open -= 1
            
            # If current node state matches goal state, success
            match = (current_node.state.state & goal_state.mask) == (goal_state.state & goal_state.mask)
            
            #print(current_node.action_name)
            #print(bin(current_node.state.state & goal_state.mask))
            #print(bin(goal_state.state & goal_state.mask))
            #print(match)
            
            if match:
                return self.reconstruct_plan(current_node, plan_size)
            
            # Add to closed
            self.closed[self.num_closed] = current_node
            self.num_closed+=1
            
            if self.num_closed == MAX_CLOSED:
                print("Closed set overflow")
                return -1
            
            # Find neighbours
            action_names = [0 for i in range(MAX_ACTIONS)]
            action_costs = [0 for i in range(MAX_ACTIONS)]
            to_states = [State() for i in range(MAX_ACTIONS)]
            num_transitions, to_states, action_names, action_costs = self.action_planner.get_possible_state_transitions(current_node.state, MAX_ACTIONS)
            
            #print("NEIGHBOURS FOUND: ")
            #print(num_transitions, action_names, action_costs)
            #print("TO STATES:")
            #for s in to_states:
            #    print(bin(s.state))
            
            for i in range(num_transitions):
                neighbour = AStarNode()
                cost = current_node.cost + action_costs[i]
                idx_o = self.open_state_idx(to_states[i])
                idx_c = self.closed_state_idx(to_states[i])
                # If neighbour is in OPEN and cost less than g(neighbour)
                if idx_o >= 0 and cost < self.open[idx_o].cost:
                    # Remove neighbour from OPEN, because new path is better
                    if self.num_open:
                        self.open[idx_o] = self.open[self.num_open-1]
                        self.num_open -= 1
                        idx_o = -1 # // BUGFIX: neighbor is no longer in OPEN, signal this so that we can re-add it.
                
                # If neighbour is in CLOSED and cost less than g(neighbour)
                if idx_c >= 0 and cost < self.closed[idx_c].cost:
                    # Remove neighbour from CLOSED
                    if self.num_closed:
                        self.closed[idx_c] = self.closed[self.num_closed-1]
                        self.num_closed -= 1
                        idx_c = -1 # // BUGFIX: neighbour is no longer in CLOSED< signal this so that we can re-add it.
                
                # If neighbour is not in OPEN and neighbour not in CLOSED
                if idx_c == -1 and idx_o == -1:
                    neighbour.state = to_states[i]
                    neighbour.cost = cost
                    neighbour.remaining_cost = self.calculate_heuristic(neighbour.state, goal_state)
                    neighbour.f = neighbour.cost + neighbour.remaining_cost
                    neighbour.action_name = action_names[i]
                    neighbour.parent_state = current_node.state
                    self.open[self.num_open] = neighbour
                    self.num_open+=1
                
                if self.num_open == MAX_OPEN:
                    print("Opened set overflow")
                    return -1
        return -1
    
        

ap = ActionPlanner()
# describe repertoire of actions
ap.set_pre_cond("scout", "armedwithgun", True)
ap.set_post_cond("scout", "enemyvisible", True)

ap.set_pre_cond("approach", "enemyvisible", True)
ap.set_post_cond("approach", "nearenemy", True)

ap.set_pre_cond("aim", "enemyvisible", True)
ap.set_pre_cond("aim", "weaponloaded", True)
ap.set_post_cond("aim", "enemylinedup", True)

ap.set_pre_cond("shoot", "enemylinedup", True)
ap.set_post_cond("shoot", "enemyalive", False)

ap.set_pre_cond("load", "armedwithgun", True)
ap.set_post_cond("load", "weaponloaded", True)

ap.set_pre_cond("detonatebomb", "armedwithbomb", True)
ap.set_pre_cond("detonatebomb", "nearenemy", True)
ap.set_post_cond("detonatebomb", "alive", False)
ap.set_post_cond("detonatebomb", "enemyalive", False)

ap.set_pre_cond("flee", "enemyvisible", True)
ap.set_post_cond("flee", "nearenemy", False)

print(ap)

# describe current world state.
fr = State() 
ap.set_state(fr, "enemyvisible", False )
ap.set_state(fr, "armedwithgun", True )
ap.set_state(fr, "weaponloaded", False )
ap.set_state(fr, "enemylinedup", False )
ap.set_state(fr, "enemyalive", True )
ap.set_state(fr, "armedwithbomb", True )
ap.set_state(fr, "nearenemy", False )
ap.set_state(fr, "alive", True )

ap.print_state(fr)
print()

# describe desired world state.
goal = State()
ap.set_state(goal, "enemyalive", False)
print('goal:', bin(goal.state), bin(goal.mask))
ap.set_state(goal, "alive", True)#// add this to avoid suicide actions in the plan.

astar_plan = AStarPlan(ap)
plan = astar_plan.plan(fr, goal, 16)

print("******** PLAN ********")
print("plan: ", plan[0])
print("steps: ", plan[2])

** Actions **
scout
approach
aim
shoot
load
detonatebomb
flee
** Atoms **
armedwithgun
enemyvisible
nearenemy
weaponloaded
enemylinedup
enemyalive
armedwithbomb
alive

armedwithgun -> True
enemyvisible -> False
nearenemy -> False
weaponloaded -> False
enemylinedup -> False
enemyalive -> True
armedwithbomb -> True
alive -> True

goal: 0b0 0b100000
******** PLAN ********
plan:  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'scout', 'load', 'aim', 'shoot']
steps:  4


Fuzzy GOAP
----------
State, goal, and action all are a little fuzzy ;)

In [ ]:
# State
# A state is a graph of the environment

# Goal
# A desired state. A goal is a constraint between nodes


# Action
# An atomic affordance that can be performed on a node
# These are defined by the designer. We wont want our bot doing
# things we have not defined. The complexity of combining these simple
# actions is at the heart of this program